In [ ]:
import requests
import json
import urllib.request
import os
import time
from bs4 import BeautifulSoup

json_open = open('/home/tensho/ymovie_datasets_csv/movie_dic.json', 'r')
movie = json.load(json_open)
result=[]

def myget(url):
    filename = url.replace('/','＿')+'.html' 
    #if os.path.isfile(filename):
    #    print('local_file')
    #    with open(filename) as f:
    #        res= f.read()
    #    f.close()
    #    return   
    if True:
        print('web_file')
        res=requests.get(url)
        if res.status_code != 200:
            time.sleep(5)
            return (res.status_code,"error")
        else:
           # f = open('/home/tensho/cache/'+filename, 'w')
           # f.write(res.text)
           # f.close() 
            time.sleep(5)
            return (res.status_code,res.text)
    
link_head="https://movies.yahoo.co.jp/movie/"

#link_head2="https://movies.yahoo.co.jp0/movie/story"

def procres(url,id,needstory=False):
    i=0
    print(url)
    while i<1:
        filename='html/'+id
        if needstory:
            filename=filename+"_story"
        filename=filename+".html"
        if os.path.isfile(filename):
            print("local_file_proc")
            return
        i+=1
        result=myget(url)
        if result[0]==200:
            f=open(filename,"w")
            f.write(result[1])
            f.close()
            return
        else:
            print("id:",id,"web status error:",result[0])
    


for i,id in enumerate(movie):
    link=link_head+id+"/"
    linkt_story=link+"story/"
    procres(link,id)
    procres(linkt_story,id,needstory=True)


In [ ]:
from pymongo import MongoClient

client=MongoClient('133.2.210.24',27017) #ip, port
db=client["meta_data"] #database name
collection=db["meta"] #collection name
import requests
from bs4 import BeautifulSoup
json_open = open('/home/tensho/ymovie_datasets_csv/movie_dic.json', 'r')
movie = json.load(json_open)

for i,id in enumerate(movie):
    dic={}
    filename='html/'+id+'.html'
    try:
        soup = BeautifulSoup(open(filename, encoding='utf-8'), "html.parser")
        title=soup.find('title').text.split(' ')[0]
        l_list=soup.find_all('tr')
        for now in l_list:
            temp_list=[]
            for now in now.text.split('\n'):
                if now != '':
                    if len(temp_list)==0:
                        temp_list=[now]
                    else:
                        temp_list.append(now)
            if temp_list[0]=='原題':
                dic['タイトル']=title
            elif temp_list[0]=='原題':
                dic['原題']=temp_list[1]
            elif temp_list[0]=='製作年度':
                dic['製作年度']=temp_list[1]
            elif temp_list[0]=='上映時間':
                dic['上映時間']=temp_list[1]
            elif temp_list[0]=='製作国':
                dic['製作国']=temp_list[1]
            elif temp_list[0]=='ジャンル':
                dic['ジャンル']=temp_list[1]
            elif temp_list[0]=='監督':
                dic['監督']=temp_list[1]
            elif temp_list[0]=='製作総指揮':
                dic['製作総指揮']=temp_list[1]
            elif temp_list[0]=='脚本':
                dic['脚本']=temp_list[1]
            elif temp_list[0]=='音楽':
                dic['音楽']=temp_list[1]
        collection.insert_one(dic)
    except FileNotFoundError as e:
        print(e)
        print(type(e))

In [ ]:
from pymongo import MongoClient
import json

client=MongoClient('133.2.210.24',27017) #ip, port
db=client["meta_data"] #database name
collection=db["story"] #collection name

json_open = open('/home/tensho/ymovie_datasets_csv/movie_dic.json', 'r')
movie = json.load(json_open)
for i,id in enumerate(movie):
    dic={}
    filename='html/'+id+'_story.html'
    try:
        soup = BeautifulSoup(open(filename, encoding='utf-8'), "html.parser")
        dic['タイトル']=soup.find('title').text.split(' ')[2]
        temp_list=[]
        k=soup.find_all('h2',class_="text-middle")
        v=soup.find_all('p',class_="text-readable")
        for i,now in enumerate(v):
            for now in now.text.split('\n'):
                if now != '':
                    if now != 'シネマトゥデイ':
                        if now != '（外部リンク）':
                            if now != 'allcinema ONLINE':
                                if i<=len(k)-1:
                                    dic[k[i].text]=now
        if len(dic.keys())>1:
            collection.insert_one(dic)
    except FileNotFoundError as e:
        print(e)
        print(type(e))